In [ ]:
import pandas as pd
import numpy as np


In [ ]:
world_food_data= pd.read_csv("../../datasets/openfoodfacts/en.openfoodfacts.org.products.tsv", sep="\t", low_memory=False)

In [ ]:
world_food_data.head()